In [1]:
import sqlite3
import pandas as pd
from pandas.core.frame import DataFrame
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

from matplotlib import pyplot as plt
from time import sleep

import matplotlib


# import gnuplotlib as gp
import numpy as np

import sys

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
import requests

In [4]:
def get_df(f_path: str = "/home/kefei/.volttron/agents/"
                       "ba3caba2-2e44-49a3-a771-eaccf6b7bc3e/sqlhistorianagent-4.0.0/data/platform.historian.sqlite"
           ) -> pd.DataFrame:
    # f_path = "/home/kefei/.volttron/agents/85736715-3354-4b70-94d7-eb2ca6e88e48/sqlhistorianagent-4.0.0/data/platform.historian.sqlite"
    con = sqlite3.connect(f_path)

    # df = pd.read_sql_query("SELECT name FROM sqlite_schema WHERE type ='table' AND name NOT LIKE 'sqlite_%'", con)
    # df.head()

    df_data = pd.read_sql_query(sql="select * from data", con=con)
    # df_data

    df_topics = pd.read_sql_query(sql="select * from topics", con=con)
    # df_topics
    con.commit()  # to make sure refresh

    df_join = pd.merge(df_data, df_topics, on='topic_id')
    # df_join.head()

    return df_join

In [35]:
df = get_df()
df.tail()

,ts,topic_id,value_string,topic_name,metadata
49763,2022-05-18T19:51:30.023592+00:00,88,2,campus/building/user-fake/SampleLong1,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
49764,2022-05-18T19:51:35.020461+00:00,88,1,campus/building/user-fake/SampleLong1,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
49765,2022-05-18T19:51:40.021101+00:00,88,7,campus/building/user-fake/SampleLong1,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
49766,2022-05-18T19:51:45.024189+00:00,88,3,campus/building/user-fake/SampleLong1,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
49767,2022-05-18T19:51:50.022036+00:00,88,2,campus/building/user-fake/SampleLong1,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"


In [75]:
# inspect here
df = get_df()
df.sort_values('ts', ascending=False).head(8)


,ts,topic_id,value_string,topic_name,metadata
74043,2022-05-19T22:26:22.740738+00:00,92,false,campus-cloud/building-cloud/RestAPI/random_bool,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""UTC""}"
74023,2022-05-19T22:26:22.740738+00:00,90,24627.769,campus-cloud/building-cloud/RestAPI/bitcoin_gbp,"{""units"": ""pound"", ""type"": ""integer"", ""tz"": ""UTC""}"
74033,2022-05-19T22:26:22.740738+00:00,91,28986.0509,campus-cloud/building-cloud/RestAPI/bitcoin_eur,"{""units"": ""euro"", ""type"": ""integer"", ""tz"": ""UTC""}"
74013,2022-05-19T22:26:22.740738+00:00,89,30135.8856,campus-cloud/building-cloud/RestAPI/bitcoin_usd,"{""units"": ""dollar"", ""type"": ""integer"", ""tz"": ""UTC""}"
74032,2022-05-19T22:26:18.823408+00:00,91,28986.0509,campus-cloud/building-cloud/RestAPI/bitcoin_eur,"{""units"": ""euro"", ""type"": ""integer"", ""tz"": ""UTC""}"
74012,2022-05-19T22:26:18.823408+00:00,89,30135.8856,campus-cloud/building-cloud/RestAPI/bitcoin_usd,"{""units"": ""dollar"", ""type"": ""integer"", ""tz"": ""UTC""}"
74042,2022-05-19T22:26:18.823408+00:00,92,false,campus-cloud/building-cloud/RestAPI/random_bool,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""UTC""}"
74022,2022-05-19T22:26:18.823408+00:00,90,24627.769,campus-cloud/building-cloud/RestAPI/bitcoin_gbp,"{""units"": ""pound"", ""type"": ""integer"", ""tz"": ""UTC""}"


In [26]:
from IPython.display import display, clear_output

In [80]:
# inspect here

topic_prfix = "campus/building/restapi/"
topic_prfix = "campus/building/user-fake/"
topic_prfix = "campus-cloud/building-cloud/"
# campus-cloud/building-cloud/RestAPI/random_bool 	
# topic_prfix = "/"

df = get_df()
df_filetered = df.sort_values('ts', ascending=False)
df_display = df_filetered[df_filetered.topic_name.str.contains(topic_prfix)].head(8)
df_display

old_index = df_display.index
old_index
# df_display.loc[old_index,:]
df_display.loc[old_index,:]

count = 0
while count < 500:
    df = get_df()
    df_filetered = df.sort_values('ts', ascending=False)
    df_display = df_filetered[df_filetered.topic_name.str.contains(topic_prfix)].head(8)
    df_display
    new_index = df_display.index.copy()
    index_diff = new_index.difference(old_index)
    old_index = new_index
    if len(index_diff) > 0:
#         print(df_filetered.loc[index_diff].head(8))
#         print(df_display)
        display(df_display.loc[index_diff, :])
    else:
        print("no new data published. Count at: ", count)
    count += 1
#     print("count", count)
    sleep(4)
    clear_output(wait=True)

no new data published. Count at:  13


KeyboardInterrupt: 